# Tutorial: Seguimiento de Experimentos con MLflow

El ciclo de vida del machine learning implica entrenar múltiples algoritmos, usar diferentes hiperparámetros y librerías, y obtener distintos resultados y modelos entrenados. Esta lección explora cómo realizar un seguimiento de esos experimentos para organizar el ciclo de vida del machine learning.

## En esta leción, tú:
* Introducirás el seguimiento de experimentos de ML con MLflow.
* Registrarás un experimento y explorarás los resultados en la Interfaz de Usuario (UI).
* Guardarás parámetros, métricas y un modelo.
* Consultarás ejecuciones pasadas de forma programática.

### El Desafío de la Organización

A lo largo del ciclo de vida del machine learning:
* Los científicos de datos prueban muchos modelos diferentes.
* Usan diversas librerías.
* Cada una con diferentes hiperparámetros.

Hacer un seguimiento de estos resultados plantea un desafío de organización, que incluye el almacenamiento de:
* Experimentos
* Resultados
* Modelos
* Artefactos suplementarios
* Código
* Versiones de los datos

## Preparación del Entorno

Antes de empezar, asegúrate de tener las librerías necesarias. Puedes instalarlas usando `pip`.

In [ ]:
# Descomenta y ejecuta la siguiente línea si no tienes las librerías instaladas
# !pip install mlflow scikit-learn pandas matplotlib hyperopt shap

Para ver tus experimentos en la interfaz de usuario de MLflow, abre una terminal, navega a la carpeta donde estás ejecutando este notebook y ejecuta el siguiente comando. Esto iniciará un servidor local y creará una carpeta `mlruns` para almacenar todos los datos de tus experimentos.

`mlflow ui`

Por defecto, la interfaz estará disponible en `http://127.0.0.1:5000`.

## Seguimiento de Experimentos con MLflow

MLflow Tracking es:
* Una API de registro específica para machine learning.
* Independiente de las librerías y entornos que realizan el entrenamiento.
* Organizada en torno al concepto de **runs (ejecuciones)**, que son ejecuciones de código de ciencia de datos.
* Las ejecuciones se agrupan en **experimentos**.
* Un servidor de MLflow puede alojar muchos experimentos.

Cada ejecución puede registrar la siguiente información:
* **Parameters (Parámetros):** Pares clave-valor de parámetros de entrada, como el número de árboles en un modelo de Random Forest.
* **Metrics (Métricas):** Métricas de evaluación como RMSE o el Área Bajo la Curva ROC.
* **Artifacts (Artefactos):** Archivos de salida arbitrarios en cualquier formato. Esto puede incluir imágenes, modelos serializados (pickled) y archivos de datos.
* **Source (Fuente):** El código que originó el experimento.

<div><img src="https://mlflow.org/docs/latest/assets/images/tracking-basics-dd24b77b7d7b32c5829e257316701801.png" style="height: 400px; margin: 20px"/></div>

### Carga de Datos y Librerías

Primero, importemos las librerías necesarias y preparemos nuestro conjunto de datos de "California Housing".

In [ ]:
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

# Cargar el dataset
housing = fetch_california_housing()
X = housing["data"]
y = housing["target"]

# Dividir los datos
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Tu Primer Experimento

Comencemos un experimento usando `mlflow.start_run()`. Dentro del bloque `with`, podemos registrar un modelo, métricas y más.

In [ ]:
# Configurar experimento
mlflow.set_experiment("Basic RF Run")

# Inicia un experimento con mlflow.start_run()
with mlflow.start_run() as run:
    # Crea el modelo, entrénalo y haz predicciones
    rf = RandomForestRegressor(random_state=42)
    rf.fit(X_train, y_train)
    predictions = rf.predict(X_test)

    # Registra el modelo usando mlflow.sklearn.log_model()
    mlflow.sklearn.log_model(rf, "random_forest_model")

    # Calcula y registra las métricas del modelo
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions)
    
    metrics = {"mse": mse, "rmse": rmse, "r2": r2}
    mlflow.log_metrics(metrics)

    # Obtén el ID del experimento y de la ejecución
    run_id = run.info.run_id
    experiment_id = run.info.experiment_id
    
    print(f"Run ID: {run_id}")
    print(f"Experiment ID: {experiment_id}")
    print(f"Dentro de una ejecución de MLflow con run_id `{run_id}` y experiment_id `{experiment_id}`")

¡Ahora ve a la interfaz de MLflow (`http://127.0.0.1:5000`) en tu navegador para ver el experimento y la ejecución que acabas de registrar!

## Parámetros, Métricas y Artefactos

¡Pero espera, hay más! En el último ejemplo, registraste el nombre de la ejecución, métricas de evaluación y tu modelo como un artefacto. Ahora vamos a registrar **parámetros**, múltiples métricas y otros artefactos, como la importancia de las características (feature importances).

Para registrar artefactos como archivos, primero debemos guardarlos en disco para que MLflow pueda luego subirlos.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import mean_absolute_error

def log_rf(experiment_id, run_name, params, X_train, X_test, y_train, y_test):
  
    with mlflow.start_run(experiment_id=experiment_id, run_name=run_name) as run:
        # Crea, entrena y predice con el modelo
        rf = RandomForestRegressor(**params)
        rf.fit(X_train, y_train)
        predictions = rf.predict(X_test)

        # Registra el modelo
        mlflow.sklearn.log_model(rf, "random_forest_model")

        # Registra los parámetros
        mlflow.log_params(params)

        # Registra las métricas
        mlflow.log_metrics({
            "rmse": np.sqrt(mean_squared_error(y_test, predictions)),
            "mse": mean_squared_error(y_test, predictions), 
            "mae": mean_absolute_error(y_test, predictions), 
            "r2": r2_score(y_test, predictions)
        })

        # Registra la importancia de las características
        importance = (pd.DataFrame(list(zip(housing.feature_names, rf.feature_importances_)), columns=["Feature", "Importance"])
                      .sort_values("Importance", ascending=False))
        
        # Guarda el artefacto en un archivo local temporalmente
        importance_path = "importance.csv"
        importance.to_csv(importance_path, index=False)
        mlflow.log_artifact(importance_path, "feature-importance")

        # Registra un gráfico de la importancia de las características
        fig, ax = plt.subplots(figsize=(10, 4))
        importance.plot.bar(ax=ax)
        plt.title("Feature Importances")
        mlflow.log_figure(fig, "feature_importances.png")

        return run.info.run_id

### Ejecutar con Diferentes Parámetros

Ahora, usemos nuestra función para ejecutar y registrar dos modelos con diferentes hiperparámetros.

In [ ]:
# Ejecución 1
params_1 = {
    "n_estimators": 100,
    "max_depth": 5,
    "random_state": 42
}
log_rf(experiment_id, "Second Run", params_1, X_train, X_test, y_train, y_test)

# Ejecución 2
params_2 = {
    "n_estimators": 1000,
    "max_depth": 10,
    "random_state": 42
}
log_rf(experiment_id, "Third Run", params_2, X_train, X_test, y_train, y_test)

## Consultando Ejecuciones Anteriores

Puedes consultar ejecuciones pasadas de forma programática para usar sus datos de vuelta en Python. La forma de hacerlo es a través de un objeto **`MlflowClient`**.

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

### Buscar y Filtrar Ejecuciones

Usemos `.search_runs()` para listar todas las ejecuciones de nuestro experimento o para filtrarlas según un criterio. La función devuelve un DataFrame de Pandas con los resultados.

In [ ]:
# Buscar todas las ejecuciones en nuestro experimento
runs_df = mlflow.search_runs(experiment_ids=[experiment_id])
print("Todas las ejecuciones:")
display(runs_df[["run_id", "metrics.rmse", "params.n_estimators", "params.max_depth"]])

# Buscar ejecuciones que cumplan una condición
filtered_runs_df = mlflow.search_runs(
    experiment_ids=[experiment_id],
    filter_string="metrics.r2 > 0.6"
)
print("\nEjecuciones con R2 > 0.6:")
display(filtered_runs_df[["run_id", "metrics.r2"]])

### Acceder a los Artefactos de una Ejecución

Podemos obtener la mejor ejecución (por ejemplo, la que tiene el menor RMSE) y ver sus artefactos.

In [ ]:
# Encontrar la mejor ejecución ordenando por RMSE
best_run = filtered_runs_df.sort_values("metrics.rmse").iloc[0]
best_run_id = best_run.run_id
print(f"Mejor Run ID: {best_run_id}")

# Listar los artefactos de la mejor ejecución
artifacts = client.list_artifacts(best_run_id)
for artifact in artifacts:
    print(f"- {artifact.path}")

### Recargar un Modelo Registrado

Finalmente, podemos recargar el modelo de nuestra mejor ejecución directamente desde MLflow para usarlo.

In [ ]:
# Recargar el modelo
model_uri = f"runs:/{best_run_id}/random_forest_model"
reloaded_model = mlflow.sklearn.load_model(model_uri)

# Usar el modelo recargado
print("\nImportancia de las características del modelo recargado:")
print(reloaded_model.feature_importances_)

## Firmas y Ejemplos de Entrada

Es una buena práctica registrar un modelo con una **firma (signature)** y un **ejemplo de entrada (input example)**. Esto permite mejores comprobaciones de esquema y facilita la integración con herramientas de despliegue automático.

* **Firma:** Es el esquema de las entradas y salidas del modelo.
* **Ejemplo de Entrada:** Son algunas filas de datos de ejemplo que el modelo espera.

In [ ]:
from mlflow.models.signature import infer_signature

with mlflow.start_run(run_name="Signature Example") as run:
    rf = RandomForestRegressor(random_state=42)
    rf_model = rf.fit(X_train, y_train)
    mse = mean_squared_error(rf_model.predict(X_test), y_test)
    mlflow.log_metric("mse", mse)

    # Infiere la firma y crea un ejemplo de entrada
    signature = infer_signature(X_train, pd.DataFrame(y_train))
    input_example = X_train[0:3]
    
    # Registra el modelo con la firma y el ejemplo de entrada
    mlflow.sklearn.log_model(
        sk_model=rf_model, 
        artifact_path="rf_model_with_signature", 
        signature=signature, 
        input_example=input_example
    )
    
    print(f"Run ID: {run.info.run_id}")

## Ejecuciones Anidadas (Nested Runs)

Las ejecuciones anidadas son una herramienta organizativa útil que permite crear una estructura de árbol con ejecuciones "padre" e "hijo". Son ideales para agrupar las diferentes pruebas de un proceso de **ajuste de hiperparámetros**.

In [ ]:
with mlflow.start_run(run_name="Nested Example Parent") as parent_run:
    print(f"Parent Run ID: {parent_run.info.run_id}")
    
    # Crea una ejecución anidada con el argumento nested=True
    with mlflow.start_run(run_name="Child 1", nested=True):
        mlflow.log_param("run_name", "child_1")

    with mlflow.start_run(run_name="Child 2", nested=True):
        mlflow.log_param("run_name", "child_2")

## Autologging

Hasta ahora, hemos registrado todo manualmente. **Autologging** permite registrar métricas, parámetros y modelos sin necesidad de llamadas explícitas a `log_`.

Simplemente llama a `mlflow.autolog()` antes de tu código de entrenamiento.

**NOTA:** Con autologging, no es necesario usar un bloque `with mlflow.start_run()`.

In [ ]:
mlflow.autolog()

rf = RandomForestRegressor(random_state=42, n_estimators=200, max_depth=8)
rf_model = rf.fit(X_train, y_train)

# Desactiva autologging para las siguientes celdas
mlflow.autolog(disable=True)

Ve a la UI de MLflow. Verás que se ha creado una nueva ejecución con todos los parámetros, métricas y el modelo registrados automáticamente.

## Ajuste de Hiperparámetros con Hyperopt

Uno de los casos de uso más comunes para las ejecuciones anidadas y el autologging es el ajuste de hiperparámetros. Usaremos la librería **Hyperopt** para encontrar el mejor modelo de Random Forest.

**Nota sobre `Trials` vs `SparkTrials`:** El notebook original usaba `SparkTrials` para afinación paralela en Databricks. En un entorno local, usaremos `Trials`, que ejecuta los trabajos de forma secuencial.

In [ ]:
from hyperopt import fmin, tpe, hp, Trials

# 1. Define la función objetivo que Hyperopt minimizará (en este caso, el MSE)
def objective(params):
    # Habilita el autologging para cada prueba de hyperopt
    mlflow.autolog(log_models=False, silent=True) # No guardamos el modelo en cada iteración para ahorrar espacio
    
    with mlflow.start_run(nested=True):
        model = RandomForestRegressor(
            n_estimators=int(params["n_estimators"]), 
            max_depth=int(params["max_depth"]), 
            min_samples_leaf=int(params["min_samples_leaf"]),
            min_samples_split=int(params["min_samples_split"]),
            random_state=42
        )
        model.fit(X_train, y_train)
        pred = model.predict(X_test)
        score = mean_squared_error(pred, y_test)

    # Hyperopt minimiza el valor de retorno ('loss')
    return score

# 2. Define el espacio de búsqueda para los hiperparámetros
search_space = {
    "n_estimators": hp.quniform("n_estimators", 100, 500, 5),
    "max_depth": hp.quniform("max_depth", 5, 20, 1),
    "min_samples_leaf": hp.quniform("min_samples_leaf", 1, 5, 1),
    "min_samples_split": hp.quniform("min_samples_split", 2, 6, 1)
}

# 3. Ejecuta Hyperopt
with mlflow.start_run(run_name="Hyperopt Tuning"):
    best_params = fmin(
        fn=objective,
        space=search_space,
        algo=tpe.suggest,
        max_evals=16,
        trials=Trials()
    )

print("Mejores parámetros encontrados:")
print(best_params)

# Desactiva autologging
mlflow.autolog(disable=True)

En la UI de MLflow, verás una ejecución padre "Hyperopt Tuning" con 16 ejecuciones hijas anidadas, una por cada combinación de hiperparámetros probada.

## Recursos Adicionales

* [Documentación de Hyperopt](http://hyperopt.github.io/hyperopt/)
* [Documentación de MLflow SHAP](https://www.mlflow.org/docs/latest/python_api/mlflow.shap.html)
* [Blog sobre ajuste de hiperparámetros con MLflow y Hyperopt](https://databricks.com/blog/2019/06/07/hyperparameter-tuning-with-mlflow-apache-spark-mllib-and-hyperopt.html)